### Create SPARK_HOME and PYLIB env var and update PATH env var

In [2]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

### Initializing Spark

Build __SparkConf__ object 

    Contains information about your application.  


Create __SparkContext__ object 
    
    Tells Spark how to access a cluster. 
    

Create __SparkSession__ object

    The entry point to programming Spark with the Dataset and DataFrame API.

    Used to create DataFrame, register DataFrame as tables and execute SQL over tables etc.

In [3]:
import findspark

In [3]:
findspark.init("/usr/local/spark/")

In [4]:
from pyspark.sql import SparkSession

In [5]:
?Spark

In [9]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("Movie Recommendation Applicationk").setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)



### Loading the dependent libraries

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.sql.functions import isnan, when, count, col, countDistinct


#### Problem Statement
 Building a  system that predicts the rating a user gives to a perticular movie.


#### Data Dictionary

Ratings Data File Structure (ratings.csv)
-----------------------------------------

All ratings are contained in the file `ratings.csv`. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

    userId, movieId, rating, timestamp

The lines within this file are ordered first by userId, then, within user, by movieId.

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.


Movies Data File Structure (movies.csv)
---------------------------------------

Movie information is contained in the file `movies.csv`. Each line of this file after the header row represents one movie, and has the following format:

    movieId, title, genres

Genres are a pipe-separated list, and are selected from the following:

* Action
* Adventure
* Animation
* Children's
* Comedy
* Crime
* Documentary
* Drama
* Fantasy
* Film-Noir
* Horror
* Musical
* Mystery
* Romance
* Sci-Fi
* Thriller
* War
* Western
* (no genres listed)


### Reading the movies and ratings data and creating a dataframe

In [7]:
## Read data and create a dataframe
ratingsData = spark.read.format("csv")\
       .option("header", "true")\
       .option("inferSchema", "true")\
       .load("file:///Users/pavantej/Desktop/SCIT/sem2/big data/20180701_Batch39_CSE7322c_Recommendation/ml-latest-small/rating_edx.csv")
    
moviesData = spark.read.format("csv")\
       .option("header","true")\
       .option("inferSchema", "true")\
       .load("file:///Users/pavantej/Desktop/SCIT/sem2/big data/20180701_Batch39_CSE7322c_Recommendation/ml-latest-small/movies.csv")
    

### Understanding Data

#### Print Schema

In [8]:
ratingsData.printSchema()
moviesData.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



#### Total number of Columns and Records

In [9]:
ratingsData.count()

1048575

In [10]:
print("No. of Columns in Ratings data= {}".format(len(ratingsData.columns)))

print('No. of Records in rating data= {}'.format(ratingsData.count()))

print("No. of Columns in movies data = {}".format(len(moviesData.columns)))

print('No. of Records in movies data= {}'.format(moviesData.count()))

No. of Columns in Ratings data= 4
No. of Records in rating data= 1048575
No. of Columns in movies data = 3
No. of Records in movies data= 27278


#### Look at first 3 row of the dataframe

In [11]:
ratingsData.show(3)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
+------+-------+------+----------+
only showing top 3 rows



In [12]:
moviesData.show(3)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows



#### Summary statistics

In [13]:
ratingsData.describe().show()

+-------+------------------+------------------+------------------+--------------------+
|summary|            userId|           movieId|            rating|           timestamp|
+-------+------------------+------------------+------------------+--------------------+
|  count|           1048575|           1048575|           1048575|             1048575|
|   mean|  3527.08612259495| 8648.988281238824|3.5292716305462175|1.0960358899987383E9|
| stddev|2018.4244255314572|19100.143880088344|1.0519187535878674|1.5948985505821717E8|
|    min|                 1|                 1|               0.5|           825499934|
|    max|              7120|            130642|               5.0|          1427764240|
+-------+------------------+------------------+------------------+--------------------+



In [14]:
moviesData.describe().show()

+-------+-----------------+--------------------+------------------+
|summary|          movieId|               title|            genres|
+-------+-----------------+--------------------+------------------+
|  count|            27278|               27278|             27278|
|   mean|59855.48057042305|                null|              null|
| stddev|44429.31469707313|                null|              null|
|    min|                1|"""Great Performa...|(no genres listed)|
|    max|           131262|  ��vegtigris (2001)|           Western|
+-------+-----------------+--------------------+------------------+



Getting the count of Distinct usersIds and movieIDs

In [15]:
print ("Number of different users: " + str(ratingsData.select('userId').distinct().count()))
print ("Number of different movies: " + str(ratingsData.select('movieId').distinct().count()))
print ("Number of different movies: " + str(moviesData.select('movieId').distinct().count()))

Number of different users: 7120
Number of different movies: 14026
Number of different movies: 27278


#### Split the data into training and test sets (20% held out for testing)

In [16]:
(trainingData,testData)=ratingsData.randomSplit([0.8,0.2])

### Model Building and Evaluation

#### ALS model params


1. numBlocks is the number of blocks the users and items will be partitioned into in order to parallelize computation (defaults to 10).
2. rank is the number of latent factors in the model (defaults to 10).
3. maxIter is the maximum number of iterations to run (defaults to 10).
4. regParam specifies the regularization parameter in ALS (defaults to 1.0).
5. implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).
6. alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0).
7. nonnegative specifies whether or not to use nonnegative constraints for least squares (defaults to false).


In [17]:
from pyspark.ml.recommendation import ALS

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als= ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy='drop')

In [18]:
model = als.fit(trainingData)

In [19]:
# Predicting on the test data


In [20]:
predictions=model.transform(testData)

In [21]:
predictions.show(50)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|  3990|    148|   4.0|1422817494|  2.042097|
|  3673|    148|   2.0| 901681963| 2.0415459|
|  6225|    148|   2.0| 842275770|  2.604641|
|  3335|    148|   5.0|1023837661|  2.040286|
|  5084|    463|   2.0| 966459084|  2.787434|
|  4162|    463|   3.0| 850913460| 2.7557917|
|  4548|    463|   2.0| 846889607| 2.9697099|
|  3428|    463|   2.0|1043020761| 3.0821939|
|  5518|    471|   4.0| 956168193| 3.4692736|
|  1460|    471|   5.0| 913416668| 3.9384558|
|  6361|    471|   3.0| 837413612| 3.3940828|
|  6176|    471|   3.0| 901469133| 3.4901278|
|  3008|    471|   2.5|1112482555| 2.3525224|
|  1889|    471|   4.0| 834209425|  4.100391|
|   332|    471|   3.0| 901760618| 2.9301324|
|  1210|    471|   3.0| 967063772|   3.12928|
|  1890|    471|   4.5|1152945560|  3.202113|
|  2062|    471|   2.5|1285095422|  3.547802|
|   860|    471|   5.0| 979800826|

### Defining the evaluator

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator=RegressionEvaluator(metricName='rmse',labelCol="rating",predictionCol="prediction")

In [13]:
?RegressionEvaluator

#### Evaluation on the test data

In [23]:
rmse=evaluator.evaluate(predictions)

In [33]:
print("RMSE Error =" + str(rmse))

RMSE Error =0.8232699598238117


In [34]:
predictions.show(100)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|  4948|    148|   3.0| 832312374| 2.3302832|
|  6225|    148|   2.0| 842275770| 2.3093112|
|  1931|    148|   2.0| 848773886| 2.5986013|
|   156|    463|   4.0|1038076799|  3.384677|
|  3179|    471|   4.0|1076914207| 3.3080032|
|  5173|    471|   4.0| 856799387| 3.5659912|
|  6361|    471|   3.0| 837413612|  2.989394|
|  5157|    471|   4.0|1291260775| 3.7524853|
|  4531|    471|   4.0| 938899399|  3.481783|
|  5030|    471|   3.0| 879420344| 3.1060998|
|  1296|    471|   5.0| 945829918| 3.5904336|
|   406|    471|   1.0| 834597354| 2.1695347|
|  1030|    471|   4.0|1007159126| 3.7809534|
|  5895|    471|   5.0|1264332812|   2.76644|
|   587|    471|   5.0|1077326007| 3.5009513|
|  4866|    471|   5.0| 940340020|  3.968884|
|  1210|    471|   3.0| 967063772| 3.1346464|
|  2090|    471|   5.0| 859420630| 3.9920585|
|  3912|    471|   1.0| 943223892|

In [30]:
#recommend movies

In [36]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)

In [41]:
userRecs.show(10)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|  1580|[[67138, 4.854030...|
|  4900|[[6375, 5.6417985...|
|  5300|[[40697, 5.950086...|
|  6620|[[727, 5.5132127]...|
|   471|[[727, 4.953483],...|
|  1591|[[727, 5.832453],...|
|  4101|[[83359, 5.112993...|
|  1342|[[727, 5.673365],...|
|  2122|[[6892, 4.855317]...|
|  2142|[[727, 5.7474823]...|
+------+--------------------+
only showing top 10 rows



In [33]:
movieRecs=model.recommendForAllItems(10)

In [42]:
movieRecs.show(10)

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   1580|[[3354, 5.0430036...|
|   4900|[[1660, 5.177846]...|
|   5300|[[4258, 5.5213346...|
|   6620|[[6402, 4.85755],...|
|   7240|[[1229, 4.7823863...|
|   7340|[[3615, 4.833582]...|
|   7880|[[6402, 4.7562876...|
|  32460|[[861, 4.945589],...|
|  54190|[[2820, 4.945897]...|
|  57370|[[3828, 4.3704834...|
+-------+--------------------+
only showing top 10 rows



In [ ]:
spark.stop()